In [7]:
#To increase cell width of ipynb
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import gzip
import cv2 as cv
import os
import numpy as np

In [ ]:
HP_NUM_UNITS1 = hp.HParam('num_units1', hp.Discrete([2048, 1024, 512]))
HP_NUM_UNITS2 = hp.HParam('num_units2', hp.Discrete([512, 256, 128]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('vgg_logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS1, HP_NUM_UNITS2, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [ ]:
# For easier disk read operation
#  MobileNetV2
# np_train_path = '../../ModelNet40/extracted_features/MobileNetV2/train.npy.gz'
# np_test_path = '../../ModelNet40/extracted_features/MobileNetV2/test.npy.gz'
# np_train_label_path = '../../ModelNet40/extracted_features/MobileNetV2/train_labels.npy'
# np_test_label_path = '../../ModelNet40/extracted_features/MobileNetV2/test_labels.npy'
# np_label_names_path = '../../ModelNet40/extracted_features/MobileNetV2/label_names.npy'

#  VGG16
np_train_path = '../../ModelNet40/extracted_features/VGG16/train.npy.gz'
np_test_path = '../../ModelNet40/extracted_features/VGG16/test.npy.gz'
np_train_label_path = '../../ModelNet40/extracted_features/VGG16/train_labels.npy'
np_test_label_path = '../../ModelNet40/extracted_features/VGG16/test_labels.npy'
np_label_names_path = '../../ModelNet40/extracted_features/VGG16/label_names.npy'

print("\n..........loading dataset from numpy files..........\n")

with gzip.GzipFile(np_train_path, "r") as f:
    x_train = np.load(f)
with gzip.GzipFile(np_test_path, "r") as f:
    x_test = np.load(f)

y_train = np.load(np_train_label_path)
y_test = np.load(np_test_label_path)
label_names = np.load(np_label_names_path)
for i, y in enumerate(y_train):
    index = np.searchsorted(label_names, y)
    y_train[i] = index

for i, y in enumerate(y_test):
    index = np.searchsorted(label_names, y)
    y_test[i] = index
    
y_train = tf.one_hot(indices=y_train, depth=len(label_names))
y_test = tf.one_hot(indices=y_test, depth=len(label_names))
# print(y_train)
# print(y_test)

In [ ]:
def train_test_model(hparams):

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS1], activation=None))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
    model.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS2], activation=None))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Activation("relu"))
    model.add(tf.keras.layers.Dense(40, activation=tf.nn.softmax))

    model.compile(
        optimizer=hparams[HP_OPTIMIZER],
        loss='categorical_crossentropy',
        metrics=['accuracy'],
)
    model.fit(x_train, y_train, epochs=50) # Run with 1 epoch to speed things up for demo purposes
    _, accuracy = model.evaluate(x_test, y_test)
    return accuracy

In [ ]:
def run(run_dir, hparams):
    
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [ ]:
session_num = 0

for num_units1 in HP_NUM_UNITS1.domain.values:
    for num_units2 in HP_NUM_UNITS2.domain.values:
        for optimizer in HP_OPTIMIZER.domain.values:
            hparams = {
              HP_NUM_UNITS1: num_units1,
              HP_NUM_UNITS2: num_units2,
              HP_OPTIMIZER: optimizer,
          } 
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('vgg_logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir vgg_logs/hparam_tuning